In [2]:
import numpy as np
import pandas as pd
import os
import pickle
import math
import pdb
from matplotlib import pyplot as plt
import importlib
from scipy.optimize import fsolve

from cartopy import crs as ccrs
from cartopy import feature as cfeature
from matplotlib import cm

import loaders
import geography
import visualization

importlib.reload(loaders)
importlib.reload(geography)
importlib.reload(visualization)
import visualization as vis

%matplotlib

Using matplotlib backend: Qt5Agg


In [3]:
## MAKE RIKS DO NOT ERASE!!! 

specs = loaders.load_density()
#specs = geography.get_coordinate_functions(specs)
#airports = loaders.load_airports(specs)
#travel = loaders.load_travel(airports)['annual']
print('finish loading')

wuhan_R0 = 3
region = 'global'
kappa = 1
infected = 1
wuhan_factor =  wuhan_R0 / airports.loc['WUH', 'density'] # R_0 / density for Wuhan

airports['R0'] = airports.density * wuhan_factor
airports['p_outbreak'] = loaders.calculate_outbreaks(airports, kappa=kappa, n=infected)
travel = loaders.augment_travel(travel, airports) 
rr = travel.query('Origin == "WUH"').sort_values('risk_ij', ascending=False)
drop = ['outgoing_total', 'risk_i','Origin', 'lower','upper','origin_lon','origin_lat',	'dest_lon',	'dest_lat']
rr = rr.drop(drop, axis=1, inplace=False).set_index('Dest', drop=False)

df = pd.read_csv('./data/airports.csv')
df = df.loc[~pd.isnull(df.iata_code)]#.set_index('iata_code')
dic = dict(zip(df.iata_code, df.municipality))
rr['municipality'] = rr.Dest.replace(dic)
dic = dict(zip(df.iata_code, df.name))
rr['name'] = rr.Dest.replace(dic)
dic = dict(zip(df.iata_code, df.iso_region))
rr['region'] = rr.Dest.replace(dic)

regions = pd.read_csv('data/regions.csv')
dic = dict(zip(regions.code, regions.name))
rr['region'] = rr.region.replace(dic)
rr.reset_index(drop=True, inplace=True)
rr.to_csv(f'riskiest_wuhan{wuhan_R0}.csv')

In [16]:
df = pd.read_csv('tables/results.csv').query('Origin == "WUH"').sort_values('risk_ij', ascending=False)
df = loaders.desc_from_iata_code(df[['Dest', 'risk_ij']], 'Dest').reset_index(drop=True)
df

,Dest,risk_ij,municipality,name,region,continent
0,PEK,0.334683,Beijing,Beijing Capital International Airport,CN-11,AS
1,CAN,0.263553,Guangzhou,Guangzhou Baiyun International Airport,CN-44,AS
2,PVG,0.250196,Shanghai,Shanghai Pudong International Airport,CN-31,AS
3,CTU,0.128522,Chengdu,Chengdu Shuangliu International Airport,CN-51,AS
4,SZX,0.114409,Shenzhen,Shenzhen Bao'an International Airport,CN-44,AS
5,TSN,0.098571,Tianjin,Tianjin Binhai International Airport,CN-12,AS
6,XIY,0.069960,Xi'an,Xi'an Xianyang International Airport,CN-61,AS
7,XMN,0.061579,Xiamen,Xiamen Gaoqi International Airport,CN-35,AS
8,HGH,0.041409,Hangzhou,Hangzhou Xiaoshan International Airport,CN-33,AS
9,NKG,0.039619,Nanjing,Nanjing Lukou Airport,CN-32,AS


In [11]:
df

,Dest,risk_ij
25687,PEK,0.334683
25662,CAN,0.263553
25688,PVG,0.250196
25665,CTU,0.128522
25694,SZX,0.114409
25699,TSN,0.098571
25705,XIY,0.069960
25706,XMN,0.061579
25671,HGH,0.041409
25684,NKG,0.039619
